# Introduction

In [ ]:
%pip install ipynao==0.6.1

In [ ]:
import ipynao
from ipywidgets import Output

import asyncio
%gui asyncio

In [ ]:
nao = ipynao.NaoRobotWidget()
nao

### Connect to Robot

In [ ]:
nao.connect("192.168.1.115", port="80")

### Call services

In [ ]:
nao

In [ ]:
tts = nao.ALTextToSpeech

In [ ]:
tts.say("how is it going?")

In [ ]:
leds = nao.service("ALLeds")

In [ ]:
leds.rasta(3)

### Retrieve data from the robot


In [ ]:
nao

In [ ]:
motion = nao.ALMotion

In [ ]:
robot_summary = motion.getSummary()

In [ ]:
print(robot_summary.result())

In [ ]:
motion.wakeUp()

In [ ]:
motion.rest()

In [ ]:
nao

In [ ]:
posture = nao.service("ALRobotPosture")

In [ ]:
posture.goToPosture("StandInit", 0.5)

In [ ]:
motion.rest()

All events occur at the same time

In [ ]:
leds.fade("AllLedsBlue", 1.0, 2)
leds.fade("AllLedsRed", 1.0, 2)
leds.fade("AllLedsGreen", 0, 2)

### Camera images

In [ ]:
nao

In [ ]:
vid = nao.ALVideoDevice

In [ ]:
resolution = 0
color_space = 11
fps = 20
sub_ID = "jupy"

cam_sub = vid.subscribeCamera(sub_ID, 0, resolution, color_space, fps)

In [ ]:
sub_name = cam_sub.result()
print(sub_name)

In [ ]:
img_future = vid.getImageRemote(sub_name)

In [ ]:
img = img_future.result()

In [ ]:
%pip install Pillow

In [ ]:
from PIL import Image, ImageOps

wid, hei = img[:2]
shape = img[:3]
image_data = img[6]
pil_img = Image.frombytes('RGB',(wid, hei), bytes(image_data))
pil_img = ImageOps.mirror(pil_img)

In [ ]:
# Display the image
pil_img

### Awaiting tasks

In [ ]:
nao

In [ ]:
animate = nao.service("ALAnimationPlayer")

In [ ]:
async def little_dance():
    await motion.wakeUp()
    await animate.run("animations/Stand/Gestures/Hey_2")
    await motion.rest()

In [ ]:
asyncio.ensure_future(little_dance())

### Error Handling

In [ ]:
nao.disconnect()
nao

In [ ]:
res = nao.service("ALLeds").rasta("gibberish")